In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# time to see if putting the data together can give me any better results than what I found in the first hour of work

In [59]:
df_pro = pd.read_csv("../../../archive/this_took_way_too_much_effort_LoL_esports.csv")
df_diamond = pd.read_csv("../../../archive/with_rates_and_spec_gods_just_better.csv")

# lets make these two dataframes

In [60]:
df_pro.drop(["blueFirstBloodAssist", "redFirstBloodAssist"], axis = 1, inplace = True)

# couple last minute fixes to only include columns each dataframe has

In [61]:
df_diamond.drop(['blueHasWukong', 'redHasWukong', 'blueHasNautilus', 'redHasNautilus', 'blueHasKaiSa',
                 'redHasKaisa', 'blueHasIrelia', 'redHasIrelia', 'blueHasSylas', 'redHasSylas', 'blueWardsPlaced',
                 'blueWardsDestroyed', 'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueAvgLevel', 
                'blueTotalJungleMinionsKilled', 'redDragons', 'redHeralds', 'redTowersDestroyed', 'redAvgLevel',
                'redTotalJungleMinionsKilled', 'redWardsPlaced','redWardsDestroyed'], axis = 1, inplace = True)

# couple last minute fixes to only include columns each dataframe has

In [62]:
df_pro.rename(columns = {"blueAvgWinrate": "bluePredWinrate", "redAvgWinrate": "redPredWinrate"}, inplace = True)

# couple last minute fixes to only include columns each dataframe has

In [63]:
df_diamond["blueCSDiff"] = df_diamond["blueCSPerMin"]*10 - df_diamond["redCSPerMin"]*10

# couple last minute fixes to only include columns each dataframe has

In [64]:
df_big_lad = pd.concat([df_pro, df_diamond]).drop(["gameId", "blueChampions", "redChampions"], axis = 1)

# lets smash these guys together and see what happens

In [65]:
df_big_lad.head()

,Unnamed: 0,blueWins,blueFirstBlood,blueTotalExperience,blueGoldDiff,blueExperienceDiff,blueCSDiff,blueKills,blueAssists,blueDeaths,redTotalExperience,redAssists,blueCSPerMin,redCSPerMin,blueGoldPerMin,redGoldPerMin,blueAvgPickrate,redAvgPickrate,bluePredWinrate,redPredWinrate
0,0,1,1.0,20459.0,1793.0,2365.0,65.0,1.0,0.0,0.0,18094.0,0.0,38.2,31.7,1629.1,1449.8,0.089224,0.147306,0.485575,0.488385
1,1,0,1.0,19210.0,759.0,171.0,-8.0,1.0,3.0,0.0,19039.0,0.0,34.0,34.8,1562.3,1486.4,0.230080,0.125320,0.503757,0.481830
2,2,0,1.0,16987.0,73.0,-1.0,-24.0,3.0,3.0,2.0,16988.0,4.0,27.5,29.9,1504.3,1497.0,0.139865,0.098864,0.492928,0.500795
3,3,0,1.0,19640.0,732.0,75.0,20.0,1.0,1.0,0.0,19565.0,0.0,38.0,36.0,1617.7,1544.5,0.121817,0.202464,0.479112,0.510519
4,4,0,0.0,20020.0,1502.0,1164.0,40.0,2.0,2.0,1.0,18856.0,1.0,36.1,32.1,1675.2,1525.0,0.166755,0.188089,0.494337,0.522972


In [66]:
X = df_big_lad.drop("blueWins", axis = 1)
y = df_big_lad["blueWins"]

# making a target and variables

In [67]:
X_train, X_test, y_train, y_test = tts(X, y, random_state = 42)

# making a training and test set

In [71]:
tree = DecisionTreeClassifier()

# lets try a decision tree model

In [72]:
params = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 2, 4, 6, 8, 10, 15, 20, 25, 30, 40, 50],
    "min_samples_split": [2, 4, 6, 8, 10, 15, 20, 25],
    "min_samples_leaf": [1, 3, 5, 7, 9, 13, 15],
    "max_features": [None, "auto", "sqrt", "log2"]
}

# give me all the parameters I can wait

In [76]:
grid_search_tree = GridSearchCV(tree, params, cv = 10, scoring = "accuracy")
grid_search_tree.fit(X_train, y_train)

# lets find the best paramaters there are for accuracy

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 4, 6, 8, 10, 15, 20, 25, 30, 40,
                                       50],
                         'max_features': [None, 'auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 3, 5, 7, 9, 13, 15],
                         'min_samples_split': [2, 4, 6, 8, 10, 15, 20, 25]},
             scoring='accuracy')

In [77]:
grid_search_tree.best_params_

# here are the best parameters to use to have the best accuracy value possible

{'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 6}

In [78]:
tree_best = DecisionTreeClassifier(criterion = "gini", max_depth = 6, max_features = "sqrt", min_samples_leaf = 3,
                                  min_samples_split = 6)
tree_best.fit(X_train, y_train)

# now let us use the decision tree using the best parameters we were given

DecisionTreeClassifier(max_depth=6, max_features='sqrt', min_samples_leaf=3,
                       min_samples_split=6)

In [79]:
tree_best.score(X_test, y_test)

# andddddddd the accuracy was worse than what I found on Monday I'm thrilled to see this!

0.7080209043399227

In [80]:
from sklearn.ensemble import RandomForestClassifier

In [81]:
forest = RandomForestClassifier()

# what about a random forest? this has got to be better right?

In [82]:
params["n_estimators"] = [100, 150, 200, 300, 400, 500]

# even more parameters because that was probably the issue last time

In [ ]:
grid_search_forest = GridSearchCV(forest, params, cv = 10, scoring = "accuracy")
grid_search_forest.fit(X_train, y_train)

# this code has been going for over 24 hours now, I did the math and just maybe used too many parameters as it is 
# looking through over 320 thousand possibilities

In [ ]:
grid_search_forest.best_params_

# wishful thinking code that maybe just maybe will run someday